#### Demo requirements
- code conda env: spglue_env.yaml
- demo runnning time: few minutes
- GPU support: True
- package version: v3

In [1]:
%matplotlib inline

import os
import torch
import pandas as pd
import scanpy as sc
from SpatialGlue import SpatialGlue

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
os.environ['R_HOME'] = '/usr/local/src/R-4.0.0'


In [4]:
file_fold = './human_lymph_node/' 
output_path = './human_lymph_node/'
adata_omics1 = sc.read_h5ad(file_fold + 'adata_RNA.h5ad')
adata_omics2 = sc.read_h5ad(file_fold + 'adata_ADT.h5ad')

adata_omics1.var_names_make_unique()
adata_omics2.var_names_make_unique()

In [36]:
from SpatialGlue.preprocess import preprocessing
data = preprocessing(adata_omics1, adata_omics2, datatype='10x')

In [37]:
import numpy as np
adata_omics1.obsm['spatial'] = np.rot90(np.rot90(np.rot90(np.array(adata_omics1.obsm['spatial'])).T).T).T
adata_omics1.obsm['spatial'][:,1] = -1*adata_omics1.obsm['spatial'][:,1]

In [5]:
model = SpatialGlue.SpatialGlue(data, datatype='10x', device=device)
# train model
output = model.train()

In [39]:
adata = adata_omics1.copy()
adata.obsm['emb_latent_omics1'] = output['emb_latent_omics1'].copy()
adata.obsm['emb_latent_omics2'] = output['emb_latent_omics2'].copy()
adata.obsm['SpatialGlue'] = output['SpatialGlue'].copy()
adata.obsm['alpha'] = output['alpha']
adata.obsm['alpha_omics1'] = output['alpha_omics1']
adata.obsm['alpha_omics2'] = output['alpha_omics2']

In [42]:
df = pd.DataFrame(data=adata.obsm['SpatialGlue'],  index=adata.obs.index)
df.to_csv(os.path.join(output_path,"joint_latent.csv"))

In [6]:
file_fold = '../../joint_bench/demo_notebook/input/spatial/Mouse_Brain/' #please replace 'file_fold' with the download path
output_path = '../../joint_bench/demo_notebook/output/spatialglue/Mouse_Brain/'

adata_omics1 = sc.read_h5ad(file_fold + 'adata_RNA.h5ad')
adata_omics2 = sc.read_h5ad(file_fold + 'adata_peaks_normalized.h5ad')

adata_omics1.var_names_make_unique()
adata_omics2.var_names_make_unique()

from SpatialGlue.preprocess import preprocessing
data = preprocessing(adata_omics1, adata_omics2, datatype='Spatial-epigenome-transcriptome')

In [7]:
model = SpatialGlue.SpatialGlue(data, datatype='Spatial-epigenome-transcriptome', device=device)

# train model
output = model.train()

adata = adata_omics1.copy()
adata.obsm['emb_latent_omics1'] = output['emb_latent_omics1']
adata.obsm['emb_latent_omics2'] = output['emb_latent_omics2']
adata.obsm['SpatialGlue'] = output['SpatialGlue']
adata.obsm['alpha'] = output['alpha']
adata.obsm['alpha_omics1'] = output['alpha_omics1']
adata.obsm['alpha_omics2'] = output['alpha_omics2']

In [33]:
df = pd.DataFrame(data=adata.obsm['SpatialGlue'],  index=adata.obs.index)
df.to_csv(os.path.join(output_path,"joint_latent.csv"))